In [1]:
# DF = returnRawDFperBenchmark(Benchmark = "is", fix="Classes", benchmarkClass = ["A", "B", "C", "D"], FixedProcess = 64, Processes = [1,2,4,8,16,32,64,128,256], FixedBenchmarkClass="B")
# DF

In [2]:
# raw_x = DF.columns.tolist()
# functionName = "MPI_Irecv()"
# raw_y = [DF.at[functionName, x] for x in raw_x]

# # グラフのプロット
# fig = plt.figure(figsize=(12, 9))
# ## 実データをsklearn用に変形
# raw_x = np.array(raw_x).reshape(-1,1)
# raw_y = np.array(raw_y).reshape(-1, 1)
# ## 説明変数と目的変数に分割
# notTrain_x, train_x, target_x = raw_x[0], raw_x[1:-1], raw_x[-1]
# notTrain_y, train_y, target_y = raw_y[0], raw_y[1:-1], raw_y[-1]
# ## 実データをそれぞれプロット
# plt.scatter(train_x, train_y, marker="o", label="予測に用いた関数コール回数")
# plt.scatter(target_x, target_y, marker="o", label="予測したい関数コール回数の実測値")
# plt.scatter(notTrain_x, notTrain_y, marker="o", label="最初のデータを除外した時に予測に用いなかった関数コール回数")
# ## モデル式をプロットするために変数”plot_x”を用意する
# plot_x_min = 1.0
# plot_x_max = 256.0
# ### モデル式としてプロットする最低値の設定
# plot_x = np.linspace(plot_x_min, plot_x_max, 500)
# plot_x = np.array(plot_x).reshape(-1, 1)
# ## モデルを [1 - 1/n] として計算
# plot_y = 1 - 1/plot_x
# plt.plot(plot_x, plot_y)


# modelIpMk2 = ModelIp_mk2(train_x=train_x, train_y=train_y, target_x=target_x, target_y=target_y)
# modelIpMk2.calc_lr()
# plot_y = modelIpMk2.predict(plot_x)
# plt.plot(plot_x, plot_y)


In [3]:
# raw_x

# raw_y

# 多変数予測でのExtra-Pへの入力ファイルを作成する

In [4]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

# 実行毎の出力は最終出力のみ
InteractiveShell.ast_node_interactivity = 'last_expr'

In [5]:
benchmarkName = "cg"
fixed = "Process"
benchmarkClasses = ["A", "B", "C", "D"]
FixedProcess = 64
Processes = [1,2,4,8,16,32,64,128,256]
FixedBenchmarkClass = "B"

def convertBenchmarkClassProcessToFload(inputDF):
    float_process = inputDF['process'].copy().astype(float)
    float_benchmarkClass = inputDF['benchmarkClass'].copy().astype(float)
    inputDF_fProcess = inputDF.copy()
    inputDF_fProcess['process'] = float_process
    inputDF_fProcess_fBenchamrkClass = inputDF_fProcess.copy()
    inputDF_fProcess_fBenchamrkClass['benchmarkClass'] = float_benchmarkClass
    return(inputDF_fProcess_fBenchamrkClass)
    

# 関数名・関数コール回数・ベンチマーククラス・ベンチマーク名・プロセス数を保持したDFを取得
DF = collectFunctionNamesPerBenchmark(benchmarkName = "cg", benchmarkClasses = benchmarkClasses, processes = Processes)
# for benchmarkClass in benchmarkClasses:
#     for Process in Processes:
#         pass

# functionNamesに先ほど取得したDFにある関数名一覧を保持
functionNames = DF["functionName"]
functionNames = functionNames[~functionNames.duplicated()]
# functionNames
# 関数ごとのデータを取得する
functionName = "VECSET"
DFperFunctionName = DF.query(f'functionName.str.endswith("{functionName}")', engine='python')
DFperFunctionName_converted = convertBenchmarkClassProcessToFload(DFperFunctionName)

rows = DFperFunctionName.index.to_list()
rows_converted = DFperFunctionName.index.to_list()
print(f"rows={rows}")
print(f"rows_converted={rows_converted}")
if(len(rows) == len(rows_converted)):
    for i in range(len(rows)):
        if (rows[i] != rows_converted[i]):
            print(f"rows[{i}]={rows[i]}, rows_converted[{i}]={rows_converted[i]}")
if(rows == rows_converted):
    print(f"same")
            
# x:ベンチマーククラス（数値化済み）
# y:プロセス数
# z:関数コール回数
DFperFunctionName_converted.sort_values(['benchmarkClass', 'process'])
x = DFperFunctionName_converted["benchmarkClass"]
y = DFperFunctionName_converted["process"]
z = DFperFunctionName_converted["call"]


rows=[12, 12, 12, 12, 12, 11, 11, 11, 11, 12, 12, 12, 13, 13, 13, 12, 12, 12, 13, 13, 13, 13, 13, 13, 12, 12, 11, 13, 13, 13, 13, 12, 13, 12, 13, 12]
rows_converted=[12, 12, 12, 12, 12, 11, 11, 11, 11, 12, 12, 12, 13, 13, 13, 12, 12, 12, 13, 13, 13, 13, 13, 13, 12, 12, 11, 13, 13, 13, 13, 12, 13, 12, 13, 12]
same


In [6]:
repeated = 3
includeLastData = False
fileName = "tmp_functionName2p.txt"

# def generateInputFile2PForExtraP(inputDF = returnRawDF(fix="Class"), repeated = 3, includeLastData = False, fileName = "tmp_functionName.txt"):
tmpfile = open(fileName, 'w+')
tmpfile.write("PARAMETER x\n")
tmpfile.write("PARAMETER y\n")

for x_num in x:
    tmpfile.write("POINTS")
    for y_num in y:
        tmpfile.write(f" ({x_num} , {y_num})")
    tmpfile.write("\n")
    
for x_num in x:
    for y_num in y:
        pass